In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeRegressor as dtr
from sklearn import tree
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier as dtc
from math import floor
import random
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error
from time import time

In [2]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import pandas as pd
from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

print("loading data")
train_data = pd.read_table("../input/train.tsv")
test_data= pd.read_table("../input/test.tsv")
print(train_data.shape)
df_tr = train_data[:100000]
df_te = test_data[:100000]

#df_tr = pd.read_table("../input/train.tsv")
#df_te = pd.read_table("../input/test.tsv")


print(df_tr.shape)
print(df_te.shape)


loading data
(1482535, 8)
(100000, 8)
(100000, 7)


In [ ]:
df_tr = df_tr.reset_index(drop=True)
df_te = df_te.reset_index(drop=True)

In [ ]:
# check for nulls
df_tr.isnull().sum()
df_te.isnull().sum()

In [ ]:
#Impute values in place of missig values.
df_tr['category_name'] = df_tr['category_name'].fillna("Unknown")
df_te['category_name'] = df_te['category_name'].fillna("Unknown")
df_tr['brand_name'] = df_tr['brand_name'].fillna("Unknown")
df_te['brand_name'] = df_te['brand_name'].fillna("Unknown")
df_tr['item_description'] = df_tr['item_description'].fillna("Unknown")
df_te['item_description'] = df_te['item_description'].fillna("Unknown")

In [ ]:
df_tr.dtypes

In [ ]:
df_tr.price.describe()

In [ ]:
df_tr.columns
df_tr.isnull().sum()

In [ ]:
df_te.isnull().sum()

In [ ]:
df_tr.head()

**Train Data Feature Engineering**

In [ ]:
#converting categorical features to meaningful quantitative features

In [ ]:
df_train = df_tr.join(df_tr['category_name'].str.split('/', expand=True).rename(columns={0:'category_1' , 1:'category_2', 2: 'category_3', 3: 'category_4'}))
df_train.shape
#df_train.columns

In [ ]:
df_train['cat1_avg'] = df_train['price'].groupby(df_train['category_1']).transform('mean')
df_train.cat1_avg = df_train.cat1_avg.round(1)
#df['cat1_rank'] = df['cat1_avg'].rank(ascending=1)
df_train['cat1_rank'] =  df_train.cat1_avg.rank(method='dense')
#df1 = df[df['category_1']=='Vintage & Collectibles']
#df1
df_train['cat2_avg'] = df_train['price'].groupby(df_train['category_2']).transform('mean')
df_train.cat2_avg = df_train.cat2_avg.round(1)
df_train['cat2_rank'] =  df_train.cat2_avg.rank(method='dense')

df_train['cat3_avg'] = df_train['price'].groupby(df_train['category_3']).transform('mean')
df_train.cat3_avg = df_train.cat3_avg.round(1)
df_train['cat3_rank'] =  df_train.cat3_avg.rank(method='dense')

df_train['cat4_avg'] = df_train['price'].groupby(df_train['category_4']).transform('mean')
df_train.cat4_avg = df_train.cat4_avg.round(1)
df_train['cat4_rank'] =  df_train.cat4_avg.rank(method='dense')

df_train['brand_avg'] = df_train['price'].groupby(df_train['brand_name']).transform('mean')
df_train.brand_avg = df_train.brand_avg.round(1)
df_train['brand_rank'] =  df_train.brand_avg.rank(method='dense')

df_train = df_train.drop([ 'category_1', 'category_2', 'category_3', 'category_4','cat1_avg', 'cat2_avg', 'cat3_avg', 'cat4_avg', 'brand_avg' ], axis=1)
df_train = df_train.drop([ 'brand_name', 'category_name',], axis=1)
df_train = df_train[['train_id','name', 'item_condition_id', 'price', 'shipping',  'item_description', 'cat1_rank',  'cat2_rank' ,  'cat3_rank' ,  'cat4_rank' ,  'brand_rank']]
df_train.head()

In [ ]:
df_train['cat1_rank'].fillna(0, inplace=True)
df_train['cat2_rank'].fillna(0, inplace=True)
df_train['cat3_rank'].fillna(0, inplace=True)
df_train['cat4_rank'].fillna(0, inplace=True)
df_train.head()

df_train.dtypes

In [ ]:
#STEMMING we have 1. porter, 2. snowball, 3. wordnetlemmatizer. Lets use wordNetLemmatizer which is not exactly as stemming, 
#the lemmatizer needs the POS(part of speech) of the word, ‘v’ (for verb), to work, and even though it is much slower than the others, 
#it returns an actual english word. 
import nltk.stem
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

#Tokenize
import re
def tokenize(text):
    stemmer = PorterStemmer()
    #lemmatizer = WordNetLemmatizer()
    tokens = []
    stems = []
    for row in text:
        rtext = re.sub("[^a-zA-Z]"," ", row)
        token = nltk.word_tokenize(rtext.lower())
        tokens.append(token)
    #print(tokens)
    for i in range(len(tokens)):
        for j in range(len(tokens[i])):
            stems.append(stemmer.stem(tokens[i][j]))
            #stems.append(lemmatizer.lemmatize(tokens[i][j], pos="n")
    return stems  

In [ ]:
t0=time()
train_name = tokenize(df_train['name'])
train_item_ds = tokenize(df_train['item_description'])
print(time()-t0)

In [ ]:
#vectorize tokens  with Tfidf, with max document frequency not higher than 10 and maximum features of 100K, using bi-gramm 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_n = TfidfVectorizer(min_df=10,max_features=100000,ngram_range=(1, 2),  stop_words='english')

In [ ]:
t0=time()
# vectorized 'name'
#X_tfidf_vectorized_n = vectorizer_n.fit_transform(df_train['name'])
vectorizer_n.fit(train_name)
train_vectorized_n = vectorizer_n.transform(df_train['name'])
vocab_tr = vectorizer_n.get_feature_names()
print(len(vocab_tr))
train_vect_n = pd.DataFrame(train_vectorized_n.toarray(), columns=vectorizer_n.get_feature_names())
#train_vect_n.head()
print(time()-t0)

In [ ]:
t0=time()
vectorizer_it = TfidfVectorizer(min_df=10,max_features=100000,ngram_range=(1, 2),  stop_words='english')
vectorizer_it.fit(train_item_ds)
train_vectorized_it = vectorizer_it.transform(df_train['item_description'])
vocab_it = vectorizer_it.get_feature_names()
train_vect_it = pd.DataFrame(train_vectorized_it.toarray(), columns=vectorizer_it.get_feature_names())
print(len(vocab_it))
print(time()-t0)

In [ ]:
t0=time()
n_comp = 20

svd = TruncatedSVD(n_components=n_comp)
s_train = svd.fit(train_vectorized_n.toarray())
train_svd = svd.transform(train_vectorized_n.toarray())
train_svd_n = pd.DataFrame(train_svd)
train_svd_n.columns = ['svd_name_'+str(i) for i in range(n_comp)]
print(time()-t0)

In [ ]:
print("original shape:   ", train_vect_n.shape)
print("transformed shape:", train_svd_n.shape)

In [ ]:
df_train_final = pd.concat([df_train, train_svd_n], axis=1)
df_train_final.shape

In [ ]:
t0=time()
n_comp = 20
svd = TruncatedSVD(n_components=n_comp)
s_train = svd.fit(train_vectorized_it.toarray())
train_svd_i = svd.transform(train_vectorized_it.toarray())
train_svd_it = pd.DataFrame(train_svd_i)
train_svd_it.columns = ['svd_it_des_'+str(i) for i in range(n_comp)]
print(time()-t0)

In [ ]:
print("original shape:   ", train_vect_it.shape)
print("transformed shape:", train_svd_it.shape)

In [ ]:
df_train_final = pd.concat([df_train_final, train_svd_it], axis=1)
df_train_final.shape

In [ ]:
df_train_f = df_train_final.drop(['name', 'item_description', 'train_id'], axis=1)
df_train_f.head()

**Test data feature engineering**

In [ ]:
df_test = df_te.join(df_te['category_name'].str.split('/', expand=True).rename(columns={0:'category_1' , 1:'category_2', 2: 'category_3', 3: 'category_4'}))
df_test.shape

In [ ]:
df_train_te = df_tr.join(df_tr['category_name'].str.split('/', expand=True).rename(columns={0:'category_1' , 1:'category_2', 2: 'category_3', 3: 'category_4'}))
df_train_te.shape

In [ ]:
df_test['cat1_avg'] = df_train_te['price'].groupby(df_train_te['category_1']).transform('mean')
df_test.cat1_avg = df_test.cat1_avg.round(1)
#df['cat1_rank'] = df['cat1_avg'].rank(ascending=1)
df_test['cat1_rank'] =  df_test.cat1_avg.rank(method='dense')
#df1 = df[df['category_1']=='Vintage & Collectibles']
#df1
df_test['cat2_avg'] = df_train_te['price'].groupby(df_train_te['category_2']).transform('mean')
df_test.cat2_avg = df_test.cat2_avg.round(1)
df_test['cat2_rank'] =  df_test.cat2_avg.rank(method='dense')

df_test['cat3_avg'] = df_train_te['price'].groupby(df_train_te['category_3']).transform('mean')
df_test.cat3_avg = df_test.cat3_avg.round(1)
df_test['cat3_rank'] =  df_test.cat3_avg.rank(method='dense')

df_test['cat4_avg'] = df_train_te['price'].groupby(df_train_te['category_4']).transform('mean')
df_test.cat4_avg = df_test.cat4_avg.round(1)
df_test['cat4_rank'] =  df_test.cat4_avg.rank(method='dense')

df_test['brand_avg'] = df_train_te['price'].groupby(df_train_te['brand_name']).transform('mean')
df_test.brand_avg = df_test.brand_avg.round(1)
df_test['brand_rank'] =  df_test.brand_avg.rank(method='dense')

df_test = df_test.drop([ 'category_1', 'category_2', 'category_3', 'category_4','cat1_avg', 'cat2_avg', 'cat3_avg', 'cat4_avg', 'brand_avg' ], axis=1)
df_test = df_test.drop([ 'brand_name', 'category_name',], axis=1)
df_test = df_test[['test_id','name', 'item_condition_id', 'shipping',  'item_description', 'cat1_rank',  'cat2_rank' ,  'cat3_rank' ,  'cat4_rank' ,  'brand_rank']]
df_test.head()

In [ ]:
df_test['cat1_rank'].fillna(0, inplace=True)
df_test['cat2_rank'].fillna(0, inplace=True)
df_test['cat3_rank'].fillna(0, inplace=True)
df_test['cat4_rank'].fillna(0, inplace=True)
df_test.head()

df_test.dtypes
df_test.shape

In [ ]:
t0=time()
test_name = tokenize(df_test['name'])
test_item_ds = tokenize(df_test['item_description'])
print(time()-t0)

In [ ]:
#vectorize tokens  with Tfidf, with max document frequency not higher than 10 and maximum features of 100K, using bi-gramm 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_n = TfidfVectorizer(min_df=10,max_features=100000,ngram_range=(1, 2),  stop_words='english')

In [ ]:
t0=time()
# vectorized 'name'
#X_tfidf_vectorized_n = vectorizer_n.fit_transform(df_train['name'])
vectorizer_n.fit(test_name)
test_vectorized_n = vectorizer_n.transform(df_test['name'])
vocab_te = vectorizer_n.get_feature_names()
print(len(vocab_te))
test_vect_n = pd.DataFrame(test_vectorized_n.toarray(), columns=vectorizer_n.get_feature_names())
#test_vect_n.head()
print(time()-t0)


In [ ]:
t0=time()
vectorizer_it = TfidfVectorizer(min_df=10,max_features=100000,ngram_range=(1, 2),  stop_words='english')
vectorizer_it.fit(test_item_ds)
test_vectorized_it = vectorizer_it.transform(df_test['item_description'])
vocab_it = vectorizer_it.get_feature_names()
test_vect_it = pd.DataFrame(test_vectorized_it.toarray(), columns=vectorizer_it.get_feature_names())
print(len(vocab_it))
print(time()-t0)

In [ ]:
t0=time()
n_comp = 20
svd = TruncatedSVD(n_components=n_comp)
s_test = svd.fit(test_vectorized_n.toarray())
test_svd = svd.transform(test_vectorized_n.toarray())
test_svd_n = pd.DataFrame(test_svd)
test_svd_n.columns = ['svd_name_'+str(i) for i in range(n_comp)]
print(time()-t0)

In [ ]:
print("original shape:   ", test_vect_n.shape)
print("transformed shape:", test_svd_n.shape)

In [ ]:
df_test_final = pd.concat([df_test, test_svd_n], axis=1)
df_test_final.shape

In [ ]:

t0=time()
n_comp = 20
svd = TruncatedSVD(n_components=n_comp)
s_test = svd.fit(test_vectorized_it.toarray())
test_svd_i = svd.transform(test_vectorized_it.toarray())
test_svd_it = pd.DataFrame(test_svd_i)
test_svd_it.columns = ['svd_it_des_'+str(i) for i in range(n_comp)]
print(time()-t0)

In [ ]:
print("original shape:   ", test_vect_it.shape)
print("transformed shape:", test_svd_it.shape)

In [ ]:
df_test_final = pd.concat([df_test_final, test_svd_it], axis=1)
df_test_final.shape

In [ ]:
df_test_f = df_test_final.drop(['name', 'item_description', 'test_id'], axis=1)
df_test_f.head()

**Train and validation data split**

In [ ]:
X= df_train_f.drop(['price'], axis = 1)
X_train,X_val,Y_train,Y_val = train_test_split(X,df_train_f['price'],test_size = 0.2)

X_train.shape

**Train a model**

In [ ]:
#Apply Desicion Trees
from sklearn.tree import DecisionTreeRegressor as dtr
regressor = dtr(random_state=0)
y_pred = regressor.fit(X_train,Y_train)
y_pred

In [ ]:
predictions = y_pred.predict(X_val)
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(Y_val, predictions))
print("RMSE for training data:", rmse)

In [ ]:
t0=time()
#Apply Random forest regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
model = RandomForestRegressor(n_estimators=1000, n_jobs=-1)
model.fit(X_train,Y_train)
print(time()-t0)

In [ ]:
print( model.feature_importances_)

In [ ]:
# Apply the Regressor we trained to the valuation data
ypred = model.predict(X_val)
print('MAE', mean_absolute_error(Y_val, ypred))
print('MSE', mean_squared_error(Y_val, ypred))
print('R2', r2_score(Y_val, ypred, multioutput='variance_weighted'))
print('M_log_error', mean_squared_log_error(Y_val, ypred))

**Apply model to test data**

In [ ]:
# Apply the Regressor we trained to the test data (which, remember, it has never seen before)
ypred_final = model.predict(df_test_f)

In [ ]:
print(df_test_final['test_id'], ypred_final)

In [ ]:
yfinal = pd.DataFrame({'price': ypred_final })
yfinal['test_id'] = df_test_final['test_id']

yfinal.to_csv("sample_submission.csv", encoding='utf-8', index=False)